In [ ]:
# use output cell to create a link to open this notebook in Google Colab
from IPython.display import Markdown
notebook_name = "tokenizer-simple.ipynb"
link = f"https://colab.research.google.com/github/ati-ozgur/course-nn-deep-learning/blob/master/notebooks/tokenizer-simple.ipynb"
Markdown(f'<a target="_blank" href="{link}">Run in Google Colab</a>')

## Yazıyı sayılaştırma (Tokenizing text)

Adapted from [LLM from scratch github repo ch02 notebook](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

In [ ]:
import os
import urllib.request

In [ ]:
if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/ati-ozgur/"
           "/course-nn-deep-learning/master/data/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [ ]:

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

In [ ]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)


r'(\s)' whitespace boşluk karakterleri ile ayrıştırma yapmamızı sağlıyor. 

In [ ]:
result = re.split(r'([,.]|\s)', text)
print(result)

r'([,.]|\s)' 
- \ boşluk
- [] içinde verilenlerden biri
- | veya anlamında

Kısaca bu regex bizim noktalama işaretleri ,. ve boşluk karakterleri ile ayrıştırma yapmamızı sağlıyor.
LLM türevleri kullanarak istediğiniz regex yazabilir veya yazılmış bir regex'in tercümesini yapabilirsiniz.

In [ ]:
# boş betikleri filtrele ve kelimeleri liste içine al
result = [item for item in result if item.strip()]
print(result)

In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

Toplam kaç simge (token) olduğunu görelim.

In [ ]:
print(len(preprocessed))

In [ ]:
# Kaç tane biricik simge
print(len(set(preprocessed)))

![](../images/tokenization-word-example.png)

In [ ]:
# sorted ile sıralama yapıyoruz
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)


In [ ]:
# dictionary comprehension ile kelime -> indeks eşlemesi
vocab = {token:integer for integer,token in enumerate(all_words)}

In [ ]:
# ilk 20 simge ve indeksleri
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


- **encode** function yazıyı sayıya çeviriyor
- **decode** function sayıları geri yazıya çeviriyor

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)



In [ ]:
tokenizer.decode(ids)

##  Special context tokens